In [1]:
import numpy as np
from class_lorenz63_mdl import LORENZ63
from class_state_vec import state_vector
from class_obs import obs_da
from class_da_sys import da_system
from sys import argv
import random as rd

#-----------------------------------------------------------------------
# Usage:
# python analysis_init.py {method}
#
# where {method} is any one of:
#  skip
#  nudging
#  OI
#  3DVar
#  ETKF
#  PF
#  Hybrid
#-----------------------------------------------------------------------

#-----------------------------------------------------------------------
# Read the Lorenz-96 nature run
#-----------------------------------------------------------------------

infile = 'lorenz63_run.pkl'
sv = state_vector()
sv = sv.load(infile)
x_nature = sv.getTrajectory()
xdim,maxit = np.shape(x_nature)


#-----------------------------------------------------------------------
# Read the Lorenz-96 observations
#-----------------------------------------------------------------------
infile = 'y_obs.pkl'
obs = obs_da()
obs = obs.load(infile)

#-----------------------------------------------------------------------
# get the observed states
#-----------------------------------------------------------------------
yp = list(range(0,3)) 

if len(yp) < xdim:
  obs.reduceDim(yp)

y_obs = obs.getVal()
y_pts = obs.getPos()
y_err = obs.getErr()

ydim, ycolumn = np.shape(y_obs)

#-----------------------------------------------------------------------
# Initialize the da system
#-----------------------------------------------------------------------
das = da_system()
das.setStateVector(sv)
das.setObsData(obs)
das.xdim = xdim
das.ydim = ydim
das.x0 = x_nature[:,0]
das.t = sv.getTimes()
das.t0 = das.t[0]

#-----------------------------------------------------------------------
# Initialize the ensemble
#-----------------------------------------------------------------------
das.edim = 3
das.ens_bias_init = 0
das.ens_sigma_init = np.sqrt(2)

#-----------------------------------------------------------------------
# Initialize the error covariances B and R, and the linearized 
# observation operator H
#-----------------------------------------------------------------------

I = np.identity(xdim)

# Set background error covariance

B = 0*I 
# Set observation error covariance

R = 2*I


# Set the linear observation operator matrix as I
H = I

const = 1.0
C = I * const

das.setB(B)
das.setR(R)
das.setH(H)
das.setC(C)

# Update the matrices to fit the reduced observation dimension
if len(yp) < xdim:
  das.reduceYdim(yp)

print('B = ')
print(das.getB())
print('R = ')
print(das.getR())
print('H = ')
print(das.getH())


#-----------------------------------------------------------------------
# Initialize the timesteps
#-----------------------------------------------------------------------
t_nature = sv.getTimes()

acyc_step = 8                       # (how frequently to perform an analysis)
dtau = (t_nature[acyc_step] - t_nature[0])
fcst_step = acyc_step                      # (may need to change for 4D DA methods)
fcst_dt = dtau / fcst_step

# Store basic timing info in das object
das.acyc_step = acyc_step
das.dtau = dtau
das.fcst_step = fcst_step
das.fcst_dt = fcst_dt
das.dt = (t_nature[1] - t_nature[0])
print('das.dt = ', das.dt)
das.maxit = maxit
das.xdim = xdim


#-----------------------------------------------------------------------
# Choose DA method:
#-----------------------------------------------------------------------

#method = argv[1]

#-----------
# Test basic functionality
#-----------
#method='skip'

#-----------
# 3D methods
#-----------
# Nudging
#method='nudging'
# OI
#method='OI'
# 3D-Var
#method='3DVar'

#-----------
# Ensemble methods
#-----------
# EnKF
method='EnKF'
# Particle filter
#method='PF'

#-----------
# 4D methods
#-----------
# 4D-Var
#method='4DVar'
# 4DEnVar
#method='4DEnVar'
# 4DETKF
#method='4DETKF'

#-----------
# Hybrid methods
#-----------
# Hybrid-Gain
#method='Hybrid'

das.setMethod(method)

#-----------------------------------------------------------------------
# Store DA object
#-----------------------------------------------------------------------
name = 'x_analysis_init'
outfile=name+'.pkl'
das.save(outfile)

print(das)


B = 
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
R = 
[[2. 0. 0.]
 [0. 2. 0.]
 [0. 0. 2.]]
H = 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
das.dt =  0.01
xdim=  3
ydim=  3
x0=  [ 8.20747939 10.0860429  23.86324441]
t0=  0.0
dt=  0.01
t= [0.000000e+00 1.000000e-02 2.000000e-02 ... 7.999997e+04 7.999998e+04
 7.999999e+04]
acyc_step= 8
dtau=  0.08
fcst_step=  8
fcst_dt= 0.01
B = 
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
H = 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
state_vector = 
lorenz63_run
Number of states and parameters
3
Parameters:
[]
Number of parameters:
0
Initial condition:
[-5.6784855   1.74220421 -0.50894036]
Number of states:
3
Trajectory:
[[ 8.20747939  8.39809589  8.59182394 ... -4.9986383  -4.69976127
  -4.4329259 ]
 [10.0860429  10.31905559 10.5401999  ... -1.85089003 -1.87383032
  -1.92237055]
 [23.86324441 24.07111655 24.31175358 ... 27.47111935 26.83733919
  26.21667603]]
EnsTrajectory:
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.